In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

# Set dataset path
DATASET_PATH = "f:/3rd year/6 sem/SE/dataset"  # Update this to your dataset location

# Define training parameters
IMG_SIZE = (299, 299)  # Xception model requires 299x299 input images
BATCH_SIZE = 32
EPOCHS = 10  # You can increase for better accuracy

# Data augmentation & preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of data used for validation
)

# Load train & validation datasets
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Load Pre-trained Xception Model
base_model = Xception(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

# Save the model for Flask
MODEL_PATH = "saved_model.h5"
model.save(MODEL_PATH)

print(f"Model saved at {MODEL_PATH}")


Found 3376 images belonging to 4 classes.
Found 841 images belonging to 4 classes.
Epoch 1/10
106/106 [==============================] - 845s 8s/step - loss: 0.7156 - accuracy: 0.7029 - val_loss: 0.7261 - val_accuracy: 0.6813
Epoch 2/10
106/106 [==============================] - 801s 8s/step - loss: 0.5526 - accuracy: 0.7805 - val_loss: 0.8250 - val_accuracy: 0.6742
Epoch 3/10
106/106 [==============================] - 792s 7s/step - loss: 0.4946 - accuracy: 0.8015 - val_loss: 0.6965 - val_accuracy: 0.7229
Epoch 4/10
106/106 [==============================] - 779s 7s/step - loss: 0.4672 - accuracy: 0.8169 - val_loss: 0.7314 - val_accuracy: 0.6849
Epoch 5/10
106/106 [==============================] - 783s 7s/step - loss: 0.4296 - accuracy: 0.8335 - val_loss: 0.6458 - val_accuracy: 0.7372
Epoch 6/10
106/106 [==============================] - 766s 7s/step - loss: 0.4181 - accuracy: 0.8368 - val_loss: 0.8203 - val_accuracy: 0.6528
Epoch 7/10
106/106 [==============================] - 752s 